In [ ]:
from ogdf_python_widget import ogdf, cppinclude
from ogdf_python_widget.widget import Widget
import ogdf_python_widget

cppinclude("ogdf/basic/graph_generators/randomized.h")
cppinclude("ogdf/layered/SugiyamaLayout.h")

cppinclude("ogdf/planarity/PlanarizationLayout.h")
cppinclude("ogdf/planarity/SubgraphPlanarizer.h")
cppinclude("ogdf/planarity/PlanarSubgraphFast.h")
cppinclude("ogdf/orthogonal/OrthoLayout.h")
cppinclude("ogdf/planarity/VariableEmbeddingInserter.h")
cppinclude("ogdf/planarity/EmbedderMinDepthMaxFaceLayers.h")
####################################################################################

G = ogdf.Graph()
ogdf.setSeed(1)
ogdf.randomPlanarTriconnectedGraph(G, 20, 40)

GA1 = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

for n in G.nodes:
    GA1.label[n] = "N%s" % n.index()

SL = ogdf.SugiyamaLayout()
SL.call(GA1)

####################################################################################

GA2 = ogdf.GraphAttributes(G, ogdf.GraphAttributes.all)

for n in G.nodes:
    GA2.label[n] = "N%s" % n.index()
    

pl = ogdf.PlanarizationLayout()

crossMin = ogdf.SubgraphPlanarizer()

ps = ogdf.PlanarSubgraphFast[int]()
ps.__python_owns__ = False
ps.runs(100)
ves = ogdf.VariableEmbeddingInserter()
ves.__python_owns__ = False
ves.removeReinsert(ogdf.RemoveReinsertType.All)

crossMin.setSubgraph(ps)
crossMin.setInserter(ves)

emb = ogdf.EmbedderMinDepthMaxFaceLayers()
emb.__python_owns__ = False
pl.setEmbedder(emb)

ol = ogdf.OrthoLayout()
ol.__python_owns__ = False
ol.separation(20.0)
ol.cOverhang(0.4)
pl.setPlanarLayouter(ol)

pl.call(GA2)

####################################################################################

G2 = ogdf.Graph()
ogdf.setSeed(2)
ogdf.randomPlanarTriconnectedGraph(G2, 19, 30)

G2.newEdge(G2.nodes[0],G2.nodes[0])

GA3 = ogdf.GraphAttributes(G2, ogdf.GraphAttributes.all)
for n in G2.nodes:
    GA3.label[n] = "N%s" % n.index()
    GA3.shape[n] = ogdf.Shape.Ellipse
    
####################################################################################

w = Widget(GA1, True)

import ipywidgets as widgets
from ipywidgets import HBox, VBox

##################################################################################

addNodeMoveToggle = widgets.Checkbox(
    value=False,
    description='move Nodes',
    disabled=False,
    indent=True
)

def on_addNodeMoveToggle(change):
    global w, addNodeMoveToggle
    w.enable_node_movement(addNodeMoveToggle.value)
addNodeMoveToggle.observe(on_addNodeMoveToggle, 'value')

##################################################################################


dropdown = widgets.Dropdown(
    options=[('GA1', GA1), ('GA2', GA2), ('GA3', GA3)],
    value=GA1,
    description='GA:',
    disabled=False,
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        if animate_toggle.value and change['old'].constGraph() is change['new'].constGraph():
            w.update_graph_attributes(change['new'])
        else:
            w.set_graph_attributes(change['new'])
        
dropdown.observe(on_change)

##################################################################################

animate_toggle = widgets.Checkbox(
    value=True,
    description='animate if possible',
    disabled=False,
    indent=True
)

##################################################################################

start_fd = widgets.Button(
    description='Start Force Layout',
    disabled=False,
    tooltip='Starts the force directed layout.',
)

def on_start_fd_button_clicked(b):
    w.start_force_directed(fix_start_position=fix_position_toggle.value)

start_fd.on_click(on_start_fd_button_clicked)

stop_fd = widgets.Button(
    description='Stop Force Layout',
    disabled=False,
    tooltip='Stops the force directed layout.',
)

def on_top_fd_button_clicked(b):
    w.stop_force_directed()

stop_fd.on_click(on_top_fd_button_clicked)

##################################################################################

fix_position_toggle = widgets.Checkbox(
    value=False,
    description='fix positions',
    disabled=False,
    indent=True
)

##################################################################################



force_directed_hbox = HBox([fix_position_toggle, start_fd, stop_fd])
ga_switch_hbox = HBox([dropdown, animate_toggle])
vbox = VBox([addNodeMoveToggle, ga_switch_hbox, force_directed_hbox, w])

vbox